In [7]:
import os
import pandas as pd
from IPython.display import display
from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch import nn
import torch

# Spécifiez le chemin du modèle DistilBERT
distilbert_model_path = r'C:\Users\vikne\Documents\Master 2\Semestre 9\Intelligence artificielle\Travel-Order-Resolver\ai\nlp\models\tokens_classification\distilbert_old'

# Charger le tokenizer et le modèle DistilBERT
try:
    tokenizer = AutoTokenizer.from_pretrained(distilbert_model_path)
    model = AutoModelForTokenClassification.from_pretrained(distilbert_model_path)
    print("Modèle DistilBERT et tokenizer chargés avec succès.")
except Exception as e:
    print(f"Erreur lors du chargement du modèle DistilBERT ou du tokenizer: {e}")

# Liste des phrases pour l'évaluation
phrases = [
    "Je veux aller de Strasbourg à Lille.",
    "Peux-tu me guider pour aller de Dijon à Clermont-Ferrand ?",
    "Je vais de Nancy à Metz en passant par Thionville.",
    "Quel est le meilleur trajet entre Poitiers et Limoges ?",
    "Je cherche une route pour aller à Reims en partant de Chartres.",
    "Il y a un train de Chambéry à Grenoble demain matin ?",
    "Je veux organiser un trajet depuis Aix-en-Provence jusqu'à Cannes.",
    "Quelle est la distance entre Rouen et Évreux ?",
    "Je vais partir de Brest pour aller à Quimper.",
    "Trouve-moi un moyen de voyager de Laval à Angers.",
    "Comment aller de Perpignan à Narbonne ?",
    "Je cherche un train de Béziers à Sète.",
    "Est-ce que je peux aller à Bayonne en partant de Pau ?",
    "Un trajet entre Montpellier et Nîmes, c'est possible ?",
    "Peux-tu calculer l'itinéraire de Nice à Monaco ?",
    "Je veux partir de Bordeaux et arriver à Arcachon.",
    "Quel est le chemin entre Annecy et Genève ?",
    "Est-ce que je peux aller à Valence depuis Lyon ?",
    "Je voyage de Marseille à Toulon.",
    "Tu peux vérifier le trajet entre Rennes et Saint-Malo ?",
    "Je vais de Besançon à Belfort.",
    "Un vol est prévu entre Genève et Lausanne demain soir.",
    "Quel est le temps de trajet entre Toulouse et Perpignan ?",
    "Trouve-moi un moyen d’aller de Paris à Orléans.",
    "Je pars de Lille et j’arrive à Amiens."
]

# Liste pour stocker les résultats
results = []

# Fonction pour afficher les prédictions avec DistilBERT
for phrase in phrases:
    try:
        # Tokenizer la phrase
        inputs = tokenizer(phrase, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            # Passer les tokens dans le modèle
            outputs = model(**inputs)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=2)

        # Obtenir les étiquettes prédits pour chaque token
        tokens = tokenizer.tokenize(tokenizer.decode(inputs['input_ids'][0]))  # Récupérer les tokens
        labels = [model.config.id2label[pred.item()] for pred in predictions[0]]

        # Ajouter les entités détectées à la liste des résultats
        for token, label in zip(tokens, labels):
            if label != 'O':  # On ignore les tokens avec le label 'O'
                results.append({"Phrase": phrase, "Entity": token, "Label": label})

    except Exception as e:
        print(f"Erreur lors du traitement de la phrase '{phrase}': {e}")

# Convertir les résultats en DataFrame
df = pd.DataFrame(results)

# Afficher le tableau
if not df.empty:
    display(df)
else:
    print("Aucune entité détectée.")

,Phrase,Entity,Label
0,Je veux aller de Strasbourg à Lille.,strasbourg,B-DEP
1,Je veux aller de Strasbourg à Lille.,lille,B-ARR
2,Peux-tu me guider pour aller de Dijon à Clermo...,dijon,B-DEP
3,Peux-tu me guider pour aller de Dijon à Clermo...,clermont-ferrand,B-ARR
4,Je vais de Nancy à Metz en passant par Thionvi...,nancy,B-DEP
5,Je vais de Nancy à Metz en passant par Thionvi...,metz,B-ARR
6,Je vais de Nancy à Metz en passant par Thionvi...,thionville,B-ARR
7,Quel est le meilleur trajet entre Poitiers et ...,poitiers,B-DEP
8,Quel est le meilleur trajet entre Poitiers et ...,limoges,B-ARR
9,Je cherche une route pour aller à Reims en par...,chartres,B-DEP


In [1]:
# import os
# import pandas as pd
# from IPython.display import display
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from torch import nn
# import torch

# # Spécifiez le chemin des deux modèles DistilBERT
# distilbert_model_1_path = r'C:\Users\vikne\Documents\Master 2\Semestre 9\Intelligence artificielle\Travel-Order-Resolver\ai\nlp\models\tokens_classification\distilbert'
# distilbert_model_2_path = r'C:\Users\vikne\Documents\Master 2\Semestre 9\Intelligence artificielle\Travel-Order-Resolver\ai\nlp\models\tokens_classification\distilbert_msi'

# # Charger les tokenizers et les modèles DistilBERT
# try:
#     tokenizer_1 = AutoTokenizer.from_pretrained(distilbert_model_1_path)
#     model_1 = AutoModelForTokenClassification.from_pretrained(distilbert_model_1_path)
#     print("Modèle distilbert_1 et tokenizer chargés avec succès.")
# except Exception as e:
#     print(f"Erreur lors du chargement du modèle distilbert_1 ou du tokenizer: {e}")

# try:
#     tokenizer_2 = AutoTokenizer.from_pretrained(distilbert_model_2_path)
#     model_2 = AutoModelForTokenClassification.from_pretrained(distilbert_model_2_path)
#     print("Modèle distilbert_2 et tokenizer chargés avec succès.")
# except Exception as e:
#     print(f"Erreur lors du chargement du modèle distilbert_2 ou du tokenizer: {e}")

# # Liste des phrases pour l'évaluation
# phrases = [
#     "Je pars de Paris et j'arrive à Marseille.",
#     "Je vais à Bordeaux en partant de Toulouse.",
#     "Mon trajet va de VILLIERS SUR LOIR à JARNY.",
#     "Nous partons de Lyon et nous arrivons à Nice.",
#     "Le départ est prévu à Lille, et l'arrivée se fera à Strasbourg.",
#     "Mon itinéraire inclut un départ de Montpellier et une arrivée à Perpignan.",
#     "Le train partira de Nantes pour arriver à Rennes.",
#     "Un vol de Genève à Zurich est prévu demain matin.",
#     "Je vais de Grenoble à Dijon en voiture.",
#     "Je voyage de Bruxelles à Amsterdam ce week-end.",
#     "L'itinéraire commence à Marseille et se termine à Toulon.",
#     "Nous prenons un train de Rouen pour aller au Havre.",
#     "Le départ sera à Limoges et l'arrivée à Poitiers.",
#     "Je quitte Monaco pour aller à Menton.",
#     "Le bus va de Caen à Cherbourg.",
#     "Il part de Bayonne et arrive à Biarritz.",
#     "Je vais de Chamonix à Annecy.",
#     "Nous roulons de Bastia à Ajaccio.",
#     "Le ferry part de Calais pour arriver à Douvres.",
#     "Je pars de Bordeaux et j'arrive à Paris.",
#     "Notre vol va de Madrid à Barcelone.",
#     "Un trajet en voiture de Milan à Rome est prévu.",
#     "Je vais à Paris en partant de Lyon."
# ]

# # Liste pour stocker les résultats
# results = []

# # Fonction pour obtenir les entités détectées avec un modèle
# def get_entities(model, tokenizer, phrase):
#     try:
#         # Tokenizer la phrase
#         inputs = tokenizer(phrase, return_tensors="pt", padding=True, truncation=True)
#         with torch.no_grad():
#             # Passer les tokens dans le modèle
#             outputs = model(**inputs)
#             logits = outputs.logits
#             predictions = torch.argmax(logits, dim=2)

#         # Obtenir les étiquettes prédits pour chaque token
#         tokens = tokenizer.tokenize(tokenizer.decode(inputs['input_ids'][0]))  # Récupérer les tokens
#         labels = [model.config.id2label[pred.item()] for pred in predictions[0]]

#         # Retourner les entités détectées
#         entities = [(token, label) for token, label in zip(tokens, labels) if label != 'O']
#         return entities
#     except Exception as e:
#         print(f"Erreur lors du traitement de la phrase '{phrase}': {e}")
#         return []

# # Tester les deux modèles et comparer les résultats
# for phrase in phrases:
#     entities_model_1 = get_entities(model_1, tokenizer_1, phrase)
#     entities_model_2 = get_entities(model_2, tokenizer_2, phrase)

#     # Créer des lignes de résultats avec les entités détectées par chaque modèle
#     for entity_1 in entities_model_1:
#         results.append({"Phrase": phrase, "Entity": entity_1[0], "Label Model 1": entity_1[1], "Label Model 2": "N/A"})
#     for entity_2 in entities_model_2:
#         results.append({"Phrase": phrase, "Entity": entity_2[0], "Label Model 1": "N/A", "Label Model 2": entity_2[1]})

# # Convertir les résultats en DataFrame
# df = pd.DataFrame(results)

# # Afficher toutes les lignes du DataFrame
# pd.set_option('display.max_rows', None)  # Afficher toutes les lignes
# if not df.empty:
#     display(df)
# else:
#     print("Aucune entité détectée.")